### Descripción del programa
Este programa sirve para el cálculo a flexión de vigas de concreto reforzado siguiendo los lineamientos de la Norma Colombiana de Construcción Sismoresistente <b>(NSR-10)</b>.

La interfaz gráfica permite ingresar los valores de diseño y ver los resultados de forma interactiva. Los resultados consisten en los refuerzos máximo y mínimo, el refuerzo balanceado, <b>el refuerzo requerido (de diseño)</b> y la deformación unitaria a tracción al momento de la falla, así como una gráfica que muestra las deformaciones unitarias de la sección al momento de la falla. También se muestra la línea límite de deformaciones unitarias (para que el diseño sea satisfactorio, la deformación unitaria a tracción no debe ser menor a este límite de 0.005).

#### ¿Cómo usar este programa? 
Ejecute la celda que contiene el código (shift + enter). Una vez se ejecuta la celda, se crean la zona gráfica y los controles interactivos. Modifique los datos a través de los controles interactivos. No modifique el código de la celda; no es necesario para usar el programa.

In [1]:
from math import sqrt
import matplotlib.pyplot as plt
import ipywidgets as wdg

def flexion_vigas(h, b, r, fc, fy, Mu):
    
    # fc: f'c del concreto (En MPa, <= 30 MPa)
    # fy: fy del acero (En MPa) ¡¡¡DEBE SER GRADO 60!!!
    # h: Peralte de la viga. Altura total de la sección (En m)
    # b: Ancho de la sección (En m)
    # r: Recubrimiento al centroide del refuerzo (En m)
    # Mu: Momento de diseño (en kN-m)
    
    phi = 0.9       # En este programa no se modificará este factor
    d = h - r       # Peralte efectivo
    
    
    # Cuantía de diseño **********************************
    Rn = (Mu/1000)/(phi*b*d**2)
    B = 2*Rn/(0.85*fc)
    
    A, rho = 0, 0.001
    if (B >= 1):
        print('-----> ¡¡¡Revisar el valor de Mu!!! <-----')
    else:
        rho = (0.85*fc/fy)*(1 - sqrt(1 - B))
        A = rho*b*d
    # ****************************************************

    # Deformación unitaria del acero en la falla *********
    a = rho*fy*d/(0.85*fc)       # Profundidad del bloque de Whitney
    c = a/0.85                   # Distancia al eje neutro desde la fibra superior
    es = (d -c)*0.003/c
    # ****************************************************

    # Cuantía mínima *************************************
    Am1 = sqrt(fc)*b*d/(4*fy)
    Am2 = 1.4*b*d/fy

    if (Am1 >= Am2):
        A_min = Am1
    else:
        A_min = Am2
    # ****************************************************

    # 0.75 de la cuantía balanceada (cuantía máx) ********
    rho_bal = (0.85*0.85*fc/fy)*(600/(600 + fy))
    A_max = 0.75*rho_bal*b*d
    A_bal = rho_bal*b*d
    # ****************************************************
    
    # Resultados de los cálculos *************************
    print('El refuerzo mínimo es {} mm^2'.format(A_min*1000**2))
    print('El refuerzo de diseño es {} mm^2'.format(A*1000**2))
    print('La deformación del acero es {}'.format(es))
    # Comprobar el mecanismo que controla la falla
    if (es < 0.005):
        print('\t-----> ¡¡Esta sección no es controlada por tensión!! <-----')
    else:
        print('\tSección controlada por tensión. Phi=0.90')
    # ********************************************
    print('El refuerzo balanceado es {} mm^2'.format(A_bal*1000**2))
    print('El refuerzo máximo es {} mm^2'.format(A_max*1000**2))
    
    
    # Gráfica de los resultados
    n = 19                             # Número de puntos
    m = d/(0.003 + es)
    x = [i*(0.003 + es)/n - es for i in range(n+1)]
    y = [m*x[i] for i in range(n+1)]

    plt.figure(figsize=(7, 7))
    plt.plot(x, y, label='Def. unitaria (mm/mm)')
    plt.plot([0, 0], [c-d, c], 'k')
    #plt.plot([-es, 0.003], [0, 0], 'k')
    plt.plot([0, 0.003], [c, c])
    plt.plot([-es, 0], [c-d, c-d])
    plt.plot([-0.005, -0.005], [c-d-0.05, c+0.05], label='Límite inferior de falla controlada por tracción (0.005)')
    plt.legend()
    plt.xlim(-es-0.001, 0.004)
    plt.ylim(c-d-0.05, c+0.05)
    plt.grid()
    plt.ylabel('Peralte de la viga (m)', size=14)
    plt.xlabel('Deformación unitaria (mm/mm)', size=14)
    plt.title('DEFORMACIÓN UNITARIA EN LA SECCIÓN', size=20)
    plt.show;
    # Fin de la función principal

# Creación de la interfaz gráfica
peralte = wdg.FloatText(description='Peralte', step=0.01, value=0.3)
ancho = wdg.FloatText(description='Ancho', step=0.01, value=0.3)
recubrimiento = wdg.FloatText(description='Recubrimiento', step=0.005, value=0.025)
concreto = wdg.FloatText(description="f'c (MPa)", step=1, value=21)
acero = wdg.FloatText(description='fy (MPa)', step=1, value=420)
M_ultimo = wdg.FloatText(description='Mu (kN-m)', step=10, value=90)

asignaciones = {
    'h': peralte,
    'b': ancho,
    'r': recubrimiento,
    'fc': concreto,
    'fy': acero,
    'Mu': M_ultimo,
}

h1 = wdg.HBox([peralte, ancho, recubrimiento])
h2 = wdg.HBox([concreto, acero, M_ultimo])
v = wdg.VBox([h1, h2])

GUI = wdg.interactive_output(flexion_vigas, asignaciones)
display(GUI, v)

Output()